# **Classificação: Grau de Risco de Doença Cardíaca - Redes Neurais**

## Dependências

In [56]:
# dataset UCI

!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

# tf, keras

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Model
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers import LeakyReLU, ReLU
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# np, Plt, seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread

# sk-learn

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print('\nTensorFlow: {}'.format(tf.__version__))
print('Numpy: {}'.format(np.__version__))


TensorFlow: 2.17.0
Numpy: 1.26.4


## Dataset

In [57]:
# fetch dataset
dados = fetch_ucirepo(id=45)

features = dados.data.features
targets = dados.data.targets

df = features
df['target'] = targets

## NAs, Split

In [58]:
df = df[~df.isin(['?'])]
df = df.dropna(axis=0)

In [59]:
# X, y split

X = df.drop(['target'], axis=1)
y =  df['target']

## Train_Test, Normalização, One Hot Encoding

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [61]:
scaler = MinMaxScaler()

In [62]:
# Normalização

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

X_train.shape

(237, 13)

In [68]:
# OHE

y_train = keras.utils.to_categorical(y_train, num_class)
y_test =  keras.utils.to_categorical(y_test, num_class)

## **Modelo Classificação Multiclass**

In [66]:
model = keras.Sequential()

# Entrada
model.add(keras.Input(shape=(13,)))

# Camada um
model.add(Dense(8, kernel_regularizer=regularizers.l2(0.001)))
model.add(LeakyReLU())
model.add(BatchNormalization())

# Camada dois
model.add(Dense(16, kernel_regularizer=regularizers.l2(0.001)))
model.add(LeakyReLU())
model.add(BatchNormalization())

# Dropout
model.add(Dropout(0.2))

# Camada tr~es
model.add(Dense(8, kernel_regularizer=regularizers.l2(0.001)))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(4, kernel_regularizer=regularizers.l2(0.001)))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1, activation='softmax'))

In [67]:
model.compile(
    loss="categorical_crossentropy",
    optimizer= keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Treinando o Modelo

history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_split=0.2,
    verbose=2,
    batch_size=10

)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


19/19 - 4s - 220ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0355
Epoch 2/50
19/19 - 0s - 11ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0355
Epoch 3/50
19/19 - 0s - 10ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0355
Epoch 4/50
19/19 - 0s - 8ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0355
Epoch 5/50
19/19 - 0s - 5ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0355
Epoch 6/50
19/19 - 0s - 5ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0355
Epoch 7/50
19/19 - 0s - 8ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0355
Epoch 8/50
19/19 - 0s - 5ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0355
Epoch 9/50
19/19 - 0s - 8ms/step - accuracy: 0.2011 - loss: 0.0355 - val_accuracy: 0.1458 - val_loss: 0.0356
Epoch 10/50
19/19 - 0s - 7

In [ ]:
# Acurácia

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia Validação')
plt.title('Acurácia por Época')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()

# Perda

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda Treino')
plt.plot(history.history['val_loss'], label='Perda Validação')
plt.title('Perda por Época')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()

In [ ]:
# Avaliando com conjunto de testes

print('\n=== Avaliação do modelo (Teste) ===\n')
results = model.evaluate(X_test, y_test, batch_size=32)

print('\nPerda (teste): {}\nAcurácia (Teste): {}'.format(round(results[0],3), round(results[1],3)))
